Importar las librerias

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

Importar el data set


In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.csv to train.csv
Saving test.csv to test.csv
User uploaded file "train.csv" with length 61194 bytes
User uploaded file "test.csv" with length 28629 bytes


In [6]:
datatrain = pd.read_csv('train.csv', index_col='PassengerId')
datatest = pd.read_csv('test.csv')


df = datatrain.drop(['Name','Ticket'], axis = 'columns')
dt = datatest.drop(['Name','Ticket','PassengerId'], axis = 'columns')

df.columns = ['Superviviente', 'clase', 'genero', 'edad', 'SibSp', 'Parch', 
              'tarifa', 'cubierta', 'puerto']
dt.columns = ['clase', 'genero', 'edad', 'SibSp', 'Parch', 
              'tarifa', 'cubierta', 'puerto']

df.genero = df.genero.map({'female' : 'mujer',
                           'male' : 'hombre'}, na_action = None)
dt.genero = dt.genero.map({'female' : 'mujer',
                           'male' : 'hombre'}, na_action = None)


## codificar la variable cubierta - error


In [7]:
def substring_in_string (value, substrings):
    for substring in substrings:
        if substring in value:
            return substring

cubiertas = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']

df.cubierta.fillna("Unknown", inplace=True)
dt.cubierta.fillna("Unknown", inplace=True)

df['cubierta'] = df ['cubierta'].map(lambda x: substring_in_string(x, cubiertas))
dt['cubierta'] = dt ['cubierta'].map(lambda x: substring_in_string(x, cubiertas))

## Codificar como factor la variable de clasificación


In [8]:
df.Superviviente = pd.Categorical(df.Superviviente)


## Convertir las variables categoricas a categorias


In [9]:
cual_col = ['clase', 'genero', 'SibSp', 'Parch', 'cubierta', 'puerto']
ordi_col = ['clase', 'SibSp', 'Parch']
cuanti_col = ['edad', 'tarifa']

for c in cual_col:
    if c in ordi_col:
        df[c] = pd.Categorical(df[c], ordered=True)
        dt[c] = pd.Categorical(dt[c], ordered=True)
    else:
        df[c] = pd.Categorical(df[c])
        dt[c] = pd.Categorical(dt[c])

df.clase = pd.Categorical(df.clase, ordered=True, categories=[3,2,1])
dt.clase = pd.Categorical(dt.clase, ordered=True, categories=[3,2,1])

df.Parch = pd.Categorical(df.Parch, ordered=True, categories=[0,1,2,3,4,5,6,9])


# PREPARAR LOS DATOS PARA EL MODELO


## Gestión de Variables

### Crear variable adulto

In [10]:
df.columns

df.loc[(df.edad <= 15), "adulto"] = "joven"
df.adulto.fillna("adulto", inplace=True)

#### En el conjunto de test


In [11]:
dt.loc[(dt.edad <= 15), "adulto"] = "joven"
dt.adulto.fillna("adulto", inplace=True)

In [12]:
df.adulto = pd.Categorical(df.adulto)
dt.adulto = pd.Categorical(dt.adulto)
cual_col.append("adulto")


## Detectar y eliminar outliers


### Detectar y eliminar outliers de la categoria cubierta T


In [13]:
df.drop(df[df.cubierta == "T"].index, axis=0, inplace=True)

df.cubierta = pd.Categorical(df.cubierta, categories=["A","B","C","D","E","F","G","Unknown"])

### Detectar y eliminar outliers de la categoria adulto


In [14]:
#### Calculo de estadisticos adultos
Q1_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.25)
Q3_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.75)
IQR_edad_adulto = Q3_edad_adulto-Q1_edad_adulto

#### Calculo de los bigotes adultos
BI_edad_adulto = Q1_edad_adulto - 1.5 * IQR_edad_adulto
BS_edad_adulto = Q3_edad_adulto + 1.5 * IQR_edad_adulto

#### Ubicación de outliers adultos
ubi_outliers_adulto = (df.edad[df.adulto == "adulto"] < BI_edad_adulto) | (df.edad[df.adulto == "adulto"] > BS_edad_adulto)
 
outliers_adulto = df[df.adulto == "adulto"][ubi_outliers_adulto]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto.index, axis=0, inplace=True)


### Detectar y eliminar outliers de la variable adultos, agrupada por genero:


In [15]:
#### Calculo de estadisticos adultos-hombres 
Q1_ed_ad_hombre = df.edad[df.adulto == "adulto"][df.genero == "hombre"].quantile(0.25)
Q3_ed_ad_hombre = df.edad[df.adulto == "adulto"][df.genero == "hombre"].quantile(0.75)
IQR_ed_ad_hombre = Q3_ed_ad_hombre-Q1_ed_ad_hombre

#### Calculo de los bigotes adultos-hombres
BI_ed_ad_hombre = Q1_ed_ad_hombre - 1.5 * IQR_ed_ad_hombre
BS_ed_ad_hombre = Q3_ed_ad_hombre + 1.5 * IQR_ed_ad_hombre

#### Ubicación de outliers adultos-Supervivientes
ubi_outliers_adulto_hombre = (df.edad[df.adulto == "adulto"][df.genero == "hombre"] < BI_ed_ad_hombre) | (df.edad[df.adulto == "adulto"][df.genero == "hombre"] > BS_ed_ad_hombre)
 
outliers_adulto_hombre = df[df.adulto == "adulto"][df.genero == "hombre"][ubi_outliers_adulto_hombre]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto_hombre.index, axis=0, inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


### Detectar y eliminar outliers de la variable adultos, agrupada por supervivientes:


In [16]:
#### Calculo de estadisticos adultos-Supervivientes 
Q1_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.25)
Q3_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.75)
IQR_ed_ad_super = Q3_ed_ad_super-Q1_ed_ad_super

#### Calculo de los bigotes adultos-Supervivientes

BI_ed_ad_super = Q1_ed_ad_super - 1.5 * IQR_ed_ad_super
BS_ed_ad_super = Q3_ed_ad_super + 1.5 * IQR_ed_ad_super

#### Ubicación de outliers adultos-Supervivientes
ubi_outliers_adulto_super = (df.edad[df.adulto == "adulto"][df.Superviviente == 1] < BI_ed_ad_super) | (df.edad[df.adulto == "adulto"][df.Superviviente == 1] > BS_ed_ad_super)

outliers_adulto_super = df[df.adulto == "adulto"][df.Superviviente == 1][ubi_outliers_adulto_super]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto_super.index, axis=0, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


### SEGUNDA ITERACION eliminar outliers de la variable adultos, agrupada por supervivientes:


In [ ]:
#### Calculo de estadisticos adultos-Supervivientes 
Q1_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.25)
Q3_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.75)
IQR_ed_ad_super = Q3_ed_ad_super-Q1_ed_ad_super

#### Calculo de los bigotes adultos-Supervivientes
BI_ed_ad_super = Q1_ed_ad_super - 1.5 * IQR_ed_ad_super
BS_ed_ad_super = Q3_ed_ad_super + 1.5 * IQR_ed_ad_super

#### Ubicación de outliers adultos-Supervivientes
ubi_outliers_adulto_super = (df.edad[df.adulto == "adulto"][df.Superviviente == 1] < BI_ed_ad_super) | (df.edad[df.adulto == "adulto"][df.Superviviente == 1] > BS_ed_ad_super)
 
outliers_adulto_super = df[df.adulto == "adulto"][df.Superviviente == 1][ubi_outliers_adulto_super]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto_super.index, axis=0, inplace=True)


### Volver a calcular el Bigote superior de adulto para edad

Q1_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.25)
Q3_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.75)
IQR_edad_adulto = Q3_edad_adulto-Q1_edad_adulto
BI_edad_adulto = Q1_edad_adulto - 1.5 * IQR_edad_adulto
BS_edad_adulto = Q3_edad_adulto + 1.5 * IQR_edad_adulto


## Tratamiento de los NA


### En el conjunto de entrenamiento


In [17]:
#### Completar los NA de hombres jovenes (masters) con la media

edad_master = datatrain.Age[datatrain.Name.str.contains("Master")]
ed_master_mean = edad_master.mean()

df.loc[(datatrain.Age.isna()) & (datatrain.Name.str.contains("Master")), "edad"] = np.ceil(ed_master_mean)


##### Los masters que quedan estaban codificados como adultos - OJO
df.loc[(df.edad <= 15) & (df.genero == "hombre"), "adulto"] = "joven"


#### Completar los NA de los adultos con la media

df.edad.fillna(df.edad.mean(), inplace=True)

### En el conjunto de test 


In [18]:
#### Hay un NAN en tarifa

dt.tarifa.fillna(dt.tarifa.mean(), inplace=True)



#### Los masters
edad_master_t = datatest.Age[datatest.Name.str.contains("Master")]
ed_master_mean_t = edad_master_t.mean()
#edad_master_t.max()

dt.loc[(datatest.Age.isna()) & (datatest.Name.str.contains("Master")), "edad"] = np.ceil(ed_master_mean_t)

##### Los masters que quedan, estaban codificados como adultos - OJO
dt.loc[(dt.edad <= 15) & (dt.genero == "hombre"), "adulto"] = "joven"


#### Completar los NA de los adultos con la media

dt.edad.fillna(dt.edad.mean(), inplace=True)

### Obtener la matriz de caracteristicas y la variable independiente


In [19]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0].values

x_t = dt.copy()

x0 = pd.concat([X,x_t], axis=0)


## Codificar variables categoricas


In [20]:
dummi_col = cual_col.copy()    

for x in ordi_col:
    dummi_col.remove(x)     
       
x0 = pd.get_dummies(x0, prefix=dummi_col, columns=dummi_col, drop_first=True)

## Dividir el conjunto de datos en entrenamiento y test


In [21]:
X = x0.iloc[0:len(df),:]
x_t = x0.iloc[len(df):,:]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0, stratify=y)


# Seleccionar los modelos


## Aplicar el modelo XGBoost



### Dividir el conjunto de datos en entrenamiento y test


In [22]:
XGB = X.copy()
XGB = np.array(XGB)
xgb_t = x_t.copy
xgb_t = np.array(xgb_t)
ygb = y.copy()
ygb = np.array(ygb)


XGB_train, XGB_test, ygb_train, ygb_test = train_test_split(XGB, ygb, test_size=0.2, 
                                                            random_state=0, stratify=ygb)

### Ajustar el clasificador al conjunto de entrenamiento


In [23]:
classi_xgb = XGBClassifier(objective='binary:logistic')

#### Primera tanda:
parametersXGB = [{'booster': ['gbtree'],
                  'eta': [0.01, 0.3, 0.6],
                  'gamma': [0.01, 0.5, 1, 5, 10],
                  'max_depth' : [4,8,12],
                  'min_child_weight' : [0.1,0.5,1,5],
                  'subsample' : [0.8],
                  'colsample_bytree' : [0.5],
#                  'sampling_method' : [],
                  'lambda' : [0.1, 0.5, 1, 10],
                  'alpha' : [0,0.5,1,5,10],
                  'scale_pos_weight' : [1,3,5]
                  }
                 ]

In [24]:
grid_search_xgb = GridSearchCV(estimator = classi_xgb,
                           param_grid= parametersXGB,
                           scoring='accuracy',
                           cv = 5,
                           n_jobs = -1)

In [ ]:
grid_search_xgb = grid_search_xgb.fit(XGB_train, ygb_train, 
                                      verbose = True,
                                      eval_metric = 'aucpr',
                                      eval_set = [(XGB_test, ygb_test)]
                                      )

In [ ]:
best_accuracy_xgb = grid_search_xgb.best_score_
print(best_accuracy_xgb)

In [ ]:
best_parameters_xgb = grid_search_xgb.best_params_
print(grid_search_xgb.best_params_)

### Predicción de los resultados con el Conjunto de Testing


In [ ]:
classi_xgb = XGBClassifier(n_estimators = 500, 
                                  criterion = "entropy",
                                  max_depth = 8,
                                  max_features = 'auto',
                                  min_samples_leaf = 2,
                                  min_samples_split = 2,
                                  random_state = 0)

classi_xgb.fit(XGB_train, ygb_train)

y_pred_xgb  = classi_xgb.predict(XGB_test)

### Elaborar una matriz de confusión


In [ ]:
cmXGB = confusion_matrix(ygb_test, y_pred_xgb)
precisionXGB = (cmXGB[0,0]+cmXGB[1,1])/cmXGB.sum()
accuracy_score(ygb_test, y_pred_xgb)


# CREAR EL OUTPUT


## Output XGBClassifier


In [ ]:
classi_xgb_out = classi_xgb.fit(XGB,ygb)

y_out_xgb = classi_xgb_out.predict(xgb_t)

y_out_xgb = pd.DataFrame(y_out_xgb, columns=["Survived"])

outXGB = pd.concat([datatest.PassengerId, y_out_xgb], axis=1)
outXGB.set_index('PassengerId', inplace=True)

#outRF.to_csv('../datasets/subm_am_py_rf_tuning_090821.csv')
